In [5]:
import os
import pandas as pd
from openpyxl import load_workbook
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt
import pickle
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy.stats as stats
import copy
import matplotlib as mpl
from PIL import Image

In [44]:
fixationDispersionPath = r'C:\DTU\PupilProcessing_MATLABscripts\PUPILS-preprocessing-pipeline-master-Helia\data_0p7_baselineQ.csv'
saccadeDispersionPath = r'C:\DTU\PupilProcessing_MATLABscripts\PUPILS-preprocessing-pipeline-master-Helia\data_out_example.csv'

df_fixation = pd.read_csv(fixationDispersionPath, header=None)
df_fixation.columns = ['timeStamp', 'x', 'y', 'pupil', 'fixation_index']
df_saccade = pd.read_csv(saccadeDispersionPath)

fixation_indexTF = [50 if element > 0 else 0 for element in df_fixation.fixation_index.tolist()]
saccade_indexTF = [50 if element == 0 else 0 for element in df_saccade.saccade_indexes.tolist()][0:-1]

acceleration = np.diff(df_saccade.velocity.tolist())

print(len(df_fixation['x']), len(fixation_indexTF), len(saccade_indexTF))

1383 1383 1383


In [45]:
# fix the saccade indexes into fixation indexes

current_fixation = 1
fixation_index_velThresh = list()

for i, index in enumerate(df_saccade.saccade_indexes.tolist()[0:-2]):
    if index > 0: # saccade
        fixation_index_velThresh.append(0)
    else:
        fixation_index_velThresh.append(current_fixation)
        if df_saccade.saccade_indexes.tolist()[i+1] > 0:
            current_fixation = current_fixation + 1
            
if df_saccade.saccade_indexes.tolist()[-2] > 0:
    fixation_index_velThresh.append(0)
else:
    fixation_index_velThresh.append(fixation_index_velThresh[-2])
        
        
print(len(saccade_indexTF), len(fixation_index_velThresh))

1383 1383


In [47]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(df_saccade.velocity, '-.')
#ax.plot(acceleration, ':')

ax.plot(fixation_indexTF, 'cyan')
ax.plot(saccade_indexTF, 'orange')

ax2 = ax.twinx()
ax2.plot(df_fixation['x'], 'green')
ax2.plot(df_fixation['y'], 'blue')

In [48]:
plt.style.use('dark_background')
im = np.array(Image.open('keyboard_baselineQ.png'), dtype=np.uint8)
fig, ax = plt.subplots(figsize = (20,15)) #, dpi = dpi)
ax.imshow(im, extent=(0, 59, 34.5, 0))
ax.axis([0, 60, 35, 0])
p = list()
label_n = 0
labelPrevious = 0        
     
colorList = ['red', 'olivedrab', 'paleturquoise', 'plum', 'maroon', 'orange', 'lightgreen', 'skyblue', 'deeppink', \
                'chocolate', 'forestgreen', 'aqua', 'blueviolet', 'brown', 'hotpink', 'tomato', 'lime', 'teal', \
                'darkmagenta', 'sandybrown', 'firebrick', 'darkcyan', 'palevioletred', 'springgreen', 'salmon', \
                'red', 'olivedrab', 'paleturquoise', 'plum', 'maroon', 'orange', 'lightgreen', 'skyblue', 'deeppink', \
                'chocolate', 'forestgreen', 'aqua', 'blueviolet', 'brown', 'hotpink', 'tomato', 'lime', 'teal', \
                'darkmagenta', 'sandybrown', 'firebrick', 'darkcyan', 'palevioletred', 'springgreen', 'salmon', \
                'red', 'olivedrab', 'paleturquoise', 'plum', 'maroon', 'orange', 'lightgreen', 'skyblue', 'deeppink', \
                'chocolate', 'forestgreen', 'aqua', 'blueviolet', 'brown', 'hotpink', 'tomato', 'lime', 'teal', \
                'darkmagenta', 'sandybrown', 'firebrick', 'darkcyan', 'palevioletred', 'springgreen', 'salmon', \
                'red', 'olivedrab', 'paleturquoise', 'plum', 'maroon', 'orange', 'lightgreen', 'skyblue', 'deeppink', \
                'chocolate', 'forestgreen', 'aqua', 'blueviolet', 'brown', 'hotpink', 'tomato', 'lime', 'teal', \
                'darkmagenta', 'sandybrown', 'firebrick', 'darkcyan', 'palevioletred', 'springgreen', 'salmon']
    
       
for i, key in enumerate(range(0,len(saccade_indexTF))):
    label = fixation_index_velThresh[key]
    if not np.isnan(df_fixation['x'][key]): # current point is not a blink
        if label == 0: # it is a saccade
            ax.plot(df_fixation['x'][key], df_fixation['y'][key], marker = 'o', color = 'white')
        else: # point is a fixation
            labelCurrent = label
            if labelCurrent != labelPrevious:
                        
                label_n = label_n + 1
                #print(i, label_n, colorList[label_n], gazePoint_cm['x'][i], gazePoint_cm['y'][i])
                ax.text(df_fixation['x'][key], df_fixation['y'][key], label_n, horizontalalignment='center',\
                        size='large', color='white', weight='semibold')
            
            ax.plot(df_fixation['x'][key], df_fixation['y'][key], marker = 'o', color = colorList[label_n])
            labelPrevious = label

plt.tight_layout()

In [49]:
plt.style.use('dark_background')
im = np.array(Image.open('keyboard_baselineQ.png'), dtype=np.uint8)
fig, ax = plt.subplots(figsize = (20,15)) #, dpi = dpi)
ax.imshow(im, extent=(0, 59, 34.5, 0))
ax.axis([0, 60, 35, 0])
p = list()
label_n = 0
labelPrevious = 0        
     
colorList = ['red', 'olivedrab', 'paleturquoise', 'plum', 'maroon', 'orange', 'lightgreen', 'skyblue', 'deeppink', \
                'chocolate', 'forestgreen', 'aqua', 'blueviolet', 'brown', 'hotpink', 'tomato', 'lime', 'teal', \
                'darkmagenta', 'sandybrown', 'firebrick', 'darkcyan', 'palevioletred', 'springgreen', 'salmon', \
                'red', 'olivedrab', 'paleturquoise', 'plum', 'maroon', 'orange', 'lightgreen', 'skyblue', 'deeppink', \
                'chocolate', 'forestgreen', 'aqua', 'blueviolet', 'brown', 'hotpink', 'tomato', 'lime', 'teal', \
                'darkmagenta', 'sandybrown', 'firebrick', 'darkcyan', 'palevioletred', 'springgreen', 'salmon', \
                'red', 'olivedrab', 'paleturquoise', 'plum', 'maroon', 'orange', 'lightgreen', 'skyblue', 'deeppink', \
                'chocolate', 'forestgreen', 'aqua', 'blueviolet', 'brown', 'hotpink', 'tomato', 'lime', 'teal', \
                'darkmagenta', 'sandybrown', 'firebrick', 'darkcyan', 'palevioletred', 'springgreen', 'salmon', \
                'red', 'olivedrab', 'paleturquoise', 'plum', 'maroon', 'orange', 'lightgreen', 'skyblue', 'deeppink', \
                'chocolate', 'forestgreen', 'aqua', 'blueviolet', 'brown', 'hotpink', 'tomato', 'lime', 'teal', \
                'darkmagenta', 'sandybrown', 'firebrick', 'darkcyan', 'palevioletred', 'springgreen', 'salmon']
    
        
for i, key in enumerate(range(0,len(saccade_indexTF))):
    label = df_fixation['fixation_index'][key]
    if not np.isnan(df_fixation['x'][key]): # current point is not a blink
        if label == 0: # it is a saccade
            ax.plot(df_fixation['x'][key], df_fixation['y'][key], marker = 'o', color = 'white')
        else: # point is a fixation
            labelCurrent = label
            if labelCurrent != labelPrevious:
                        
                label_n = label_n + 1
                #print(i, label_n, colorList[label_n], gazePoint_cm['x'][i], gazePoint_cm['y'][i])
                ax.text(df_fixation['x'][key], df_fixation['y'][key], label_n, horizontalalignment='center',\
                        size='large', color='white', weight='semibold')
            
            ax.plot(df_fixation['x'][key], df_fixation['y'][key], marker = 'o', color = colorList[label_n])
            labelPrevious = label

plt.tight_layout()